In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

Using backend: pytorch


In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../../data/dataset/',
                                 transform=T.ToSparseTensor())
dataset

PygNodePropPredDataset()

In [3]:
torch.cuda.is_available()

True

In [4]:
data = dataset[0]
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [5]:
split_idx = dataset.get_idx_split()
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [6]:
data.adj_t = data.adj_t.to_symmetric()
data.adj_t

SparseTensor(row=tensor([     0,      0,      0,  ..., 169341, 169342, 169342]),
             col=tensor([   411,    640,   1162,  ..., 163274,  27824, 158981]),
             size=(169343, 169343), nnz=2315598, density=0.01%)

In [7]:
row, col, _ = data.adj_t.coo()

In [8]:
edge_index = torch.stack([col, row], dim=0)
edge_index

tensor([[   411,    640,   1162,  ..., 163274,  27824, 158981],
        [     0,      0,      0,  ..., 169341, 169342, 169342]])

In [9]:
edge_index.t()

tensor([[   411,      0],
        [   640,      0],
        [  1162,      0],
        ...,
        [163274, 169341],
        [ 27824, 169342],
        [158981, 169342]])

In [10]:
edge_index.shape

torch.Size([2, 2315598])

In [11]:
edge_index[:, 0]

tensor([411,   0])

In [12]:
edge_index.t()[0]

tensor([411,   0])

In [13]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self):
        super(LinkPredictor, self).__init__()

    def forward(self, x_i, x_j):
        x = torch.sum(torch.mul(x_i, x_j), dim=1)
        x = torch.clamp(x, max=6, min=-6)
        return torch.sigmoid(x)

In [14]:
device=0
log_steps=1
num_layers=2
hidden_channels=256
dropout=0.5
batch_size=128 * 1024
lr=0.005
epochs=100
eval_steps=2
runs=10

In [15]:
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

In [16]:
adj_t = data.adj_t.to(device)

In [17]:
torch.manual_seed(12345)

In [18]:
model = SAGE(hidden_channels, hidden_channels,
             hidden_channels, num_layers,
             dropout).to(device)

In [19]:
emb = torch.nn.Embedding(data.num_nodes, hidden_channels).to(device)

In [20]:
predictor = LinkPredictor().to(device)

In [21]:
emb.weight.size(0), emb.weight.size(1)

(169343, 256)

In [22]:
torch.nn.init.xavier_uniform_(emb.weight)

Parameter containing:
tensor([[ 0.0038, -0.0002,  0.0026,  ...,  0.0018, -0.0008, -0.0005],
        [ 0.0059, -0.0012,  0.0038,  ...,  0.0047,  0.0054, -0.0025],
        [ 0.0016, -0.0022,  0.0025,  ..., -0.0029,  0.0044, -0.0032],
        ...,
        [-0.0018, -0.0047,  0.0046,  ..., -0.0017,  0.0026, -0.0014],
        [ 0.0039,  0.0039, -0.0056,  ...,  0.0036,  0.0009, -0.0018],
        [ 0.0030,  0.0052, -0.0015,  ..., -0.0024,  0.0019,  0.0044]],
       device='cuda:0', requires_grad=True)

In [23]:
model.reset_parameters()

In [24]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(emb.parameters()) +
    list(predictor.parameters()), lr=lr)

In [25]:
def train(model, predictor, x, adj_t, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = edge_index.t().to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='sparse')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

In [26]:
for epoch in range(1, 1 + epochs):
    loss = train(model, predictor, emb.weight, adj_t,
                 optimizer, batch_size)
    
    if epoch % eval_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}')

Epoch: 02, Loss: 0.9802
Epoch: 04, Loss: 0.8893
Epoch: 06, Loss: 0.8432
Epoch: 08, Loss: 0.8143
Epoch: 10, Loss: 0.7979
Epoch: 12, Loss: 0.7886
Epoch: 14, Loss: 0.7812
Epoch: 16, Loss: 0.7756
Epoch: 18, Loss: 0.7713
Epoch: 20, Loss: 0.7671
Epoch: 22, Loss: 0.7632
Epoch: 24, Loss: 0.7574
Epoch: 26, Loss: 0.7670
Epoch: 28, Loss: 0.7257
Epoch: 30, Loss: 0.6857
Epoch: 32, Loss: 0.6401
Epoch: 34, Loss: 0.6157
Epoch: 36, Loss: 0.6048
Epoch: 38, Loss: 0.5892
Epoch: 40, Loss: 0.5766
Epoch: 42, Loss: 0.5620
Epoch: 44, Loss: 0.5584
Epoch: 46, Loss: 0.5444
Epoch: 48, Loss: 0.5364
Epoch: 50, Loss: 0.5299
Epoch: 52, Loss: 0.5213
Epoch: 54, Loss: 0.5157
Epoch: 56, Loss: 0.5107
Epoch: 58, Loss: 0.5060
Epoch: 60, Loss: 0.4997
Epoch: 62, Loss: 0.4990
Epoch: 64, Loss: 0.4892
Epoch: 66, Loss: 0.4858
Epoch: 68, Loss: 0.4788
Epoch: 70, Loss: 0.4729
Epoch: 72, Loss: 0.4675
Epoch: 74, Loss: 0.4618
Epoch: 76, Loss: 0.4614
Epoch: 78, Loss: 0.4562
Epoch: 80, Loss: 0.4551
Epoch: 82, Loss: 0.4470
Epoch: 84, Loss:

In [27]:
model.eval()

SAGE(
  (convs): ModuleList(
    (0): SAGEConv(256, 256)
    (1): SAGEConv(256, 256)
  )
)

In [28]:
emb.weight

Parameter containing:
tensor([[ 0.0621, -0.3993, -0.2266,  ...,  0.0243,  0.2232, -0.0785],
        [-0.0889,  0.0978,  0.0153,  ...,  0.1448,  0.1571,  0.1532],
        [-0.0856, -0.0886, -0.1970,  ...,  0.0314, -0.1651,  0.1031],
        ...,
        [ 0.4462,  0.3250,  0.2302,  ..., -0.0585,  0.0564, -0.2859],
        [ 0.1844,  0.0400, -0.3168,  ...,  0.1558,  0.0770, -0.0720],
        [-0.0154, -0.0099, -0.2226,  ...,  0.0221,  0.0191,  0.0860]],
       device='cuda:0', requires_grad=True)

In [29]:
h = model(emb.weight, adj_t)

In [30]:
predictor(h[0:2], h[411:413])

tensor([0.9975, 0.0025], device='cuda:0', grad_fn=<SigmoidBackward>)

In [31]:
predictor(h[0:2], h[640:642])

tensor([0.9975, 0.0025], device='cuda:0', grad_fn=<SigmoidBackward>)

In [32]:
np.save('../../models/graphsage_link_pred/full_cos_graphsage_linkpred_11_h.npy', h.cpu().detach().numpy())

In [33]:
h_new = h.cpu().detach()

In [34]:
predictor(h_new[0:2], h_new[411:413])

tensor([0.9975, 0.0025])

In [35]:
torch.save(model, '../../models/graphsage_link_pred/11_model.pt')

In [36]:
torch.save(emb, '../../models/graphsage_link_pred/11_embedding.pt')